In [1]:
import numpy as np
import nashpy as nash

def find_the_equilibrium(g):
# Function that returns the one optimal equilibrium using different methods from nashpy
	arr, arr_1 = [], []

	lhs = g.lemke_howson_enumeration()
	for lh in lhs:
		arr.append(lh)

	# Check if lemke_howson's output is is all the same (only first 2 outputs for simplicity)
	if np.array_equal(arr[0], arr[1]):
		# If so then that is the corret equlibrium
		return arr[0]

	else: # Return 3rd output of the support_enumeration method 
		#(TODO: check if it's the same with 2nd output of vertex_enumeration (it should be))
		eqs = g.support_enumeration()
		for eq in eqs:
			arr_1.append(eq)
		return arr_1[-1]

In [2]:
data = {} # for now the data is stored in a dict (TODO: laod the data into a .csv file)
# and alter simply run something like this
# for _ in range(1):
#     data.append(experiment())

In [11]:
def epic_vertic(principal_game, agents_game):
    # for measuring vertical alignment

    principal_row = principal_game.payoff_matrices[0]
    principal_col = principal_game.payoff_matrices[1]

    agent_row = agents_game.payoff_matrices[0]
    agent_col = agents_game.payoff_matrices[1]

    # note: uniform distribution assumed
    row_matrix_coeffs = np.corrcoef(
        [principal_row.flatten(), agent_row.flatten()])
    col_matrix_coeffs = np.corrcoef(
        [principal_col.flatten(), agent_col.flatten()])
    return (row_matrix_coeffs[0][1], col_matrix_coeffs[0][1])

In [47]:
def experiment(game=None, epsilon_r=None, epsilon_c=None):

	"""
	TODO: Add an ability to pass v_alignment and h_alignment inot the function!

	When passing games make sure the the pay-offs are normalize (e.g. thought an affine
	transformation) into a range [0, 1].

	principals_game is fully random (both row and column pay-offs are randomly generated)
	"""

    # (1) Sample G and G' from uniform distribution [-1, 1]
	if game is None:
		agents_game = nash.Game(np.random.uniform(-1, 1, size=(2,2)), np.random.uniform(-1, 1, size=(2,2)))
		principals_game = nash.Game(np.random.uniform(-1, 1, size=(2,2)), np.random.uniform(-1, 1, size=(2,2)))

    # (2) Sample epsilons (and a random strategy) from a uniform distribution [0, 1], 
	# where 1 playing NE and 0 is playing at random. 
	# note that when playing at random it's possible that the agent will end up playing NE (or well).
	if epsilon_c is None:
		epsilon_c = np.random.uniform(0, 1)
	if epsilon_r is None:
		epsilon_r = np.random.uniform(0, 1)
	random_strategy = np.random.uniform(0, 1)
		
	# (3) Compute Game's Equilibrium (both pure and mixed) and define the best strategies
	strategies = find_the_equilibrium(agents_game)
	row_strat = strategies[0]
	col_strat = strategies[1]
	
	# (4) Adjust strategies by epsilon
	row_strat = row_strat * epsilon_r + random_strategy * (1 - epsilon_r)
	col_strat = col_strat * epsilon_c + random_strategy * (1 - epsilon_c)
	adjusted_strategies = [row_strat, col_strat]

    # (5) Compute Vertical and Horizontal Alignment, and Principal's Welfare
	# Horizontal alignment 
	row_payoffs = agents_game.payoff_matrices[0]
	col_payoffs = agents_game.payoff_matrices[1]
	matrix_coeffs = np.corrcoef([row_payoffs.flatten(), col_payoffs.flatten()])
	h_alignment = matrix_coeffs[0][1]

	# Vertical alignemnt
	v_alignment = epic_vertic(principals_game, agents_game)

	# Principal's Welfare (Calculate the utilities for the Principal given Agents strategies)
	principal_welf = principals_game[adjusted_strategies]

	return principal_welf, epsilon_r, epsilon_c, h_alignment, v_alignment
	

experiment()

(array([-0.08869876, -0.43861364]), -0.07522328769855224)

In [ ]:
# debugging equilibrium search using nashpy methodes

game = nash.Game(np.random.uniform(-1, 1, size=(2,2)), np.random.uniform(-1, 1, size=(2,2))) # just for the testing
print(game)
print("")

print("----------vertex_enumeration()---------")
eqs = game.vertex_enumeration()
for eq in eqs:
    print(eq)

print("")

print("----------support_enumeration()---------")
ggs = game.support_enumeration()
for gg in ggs:
    print(gg)

print("")

print("----------lemke_howson_enumeration()---------")
lkms = game.lemke_howson_enumeration()
for lkm in lkms:
    print(lkm)

print("")

# Checking the output of the function
print("----------find_the_equilibrium()---------")
find_the_equilibrium(game)